In [6]:
from hypergraphx.core.hypergraph import Hypergraph

def build_hypergraph_hgx() -> Hypergraph:
    hg = Hypergraph()
    # 8 nodes
    hg.add_nodes(list("ABCDEFGH"))
    # 5 hyperedges (mixed sizes, overlapping allowed)
    hg.add_edges([
        ("A","B","C"),
        ("B","C","D"),
        ("A","D"),
        ("E","F","G"),
        ("A","E","H"),
    ])
    return hg


In [7]:
H = build_hypergraph_hgx()

In [8]:
from typing import Dict, List
from hypergraphx.core.hypergraph import Hypergraph

def edge_size_distribution(hg: Hypergraph) -> Dict[int, int]:
    # HGX returns a Counter-like mapping; normalize to plain dict (sorted for determinism).
    return dict(sorted(hg.distribution_sizes().items()))

def node_degrees(hg: Hypergraph) -> Dict[str, int]:
    return {str(u): hg.degree(u) for u in hg.get_nodes()}

def top_k_by_degree(hg: Hypergraph, k: int) -> List[str]:
    deg = node_degrees(hg)
    return [u for u, _ in sorted(deg.items(), key=lambda x: (-x[1], x[0]))[:k]]

In [9]:
print(edge_size_distribution(H))
print(node_degrees(H))
print(top_k_by_degree(H, 3))

{2: 1, 3: 4}
{'A': 3, 'B': 2, 'C': 2, 'D': 2, 'E': 2, 'F': 1, 'G': 1, 'H': 1}
['A', 'B', 'C']


In [10]:
from typing import List, Set, Tuple
from hypergraphx.core.hypergraph import Hypergraph

def incident_edges(hg: Hypergraph, node: str) -> List[Tuple[str, ...]]:
    # Convert each incident edge to a sorted tuple; return a globally sorted list.
    edges = [tuple(sorted(map(str, e))) for e in hg.get_incident_edges(node)]
    return sorted(edges)

def neighbors_by_size(hg: Hypergraph, node: str, size: int) -> Set[str]:
    return set(map(str, hg.get_neighbors(node, size=size)))

def largest_component_size(hg: Hypergraph) -> int:
    return hg.largest_component_size()

In [11]:
print(incident_edges(H, "A"))
print(neighbors_by_size(H, "A", size=3))
print(largest_component_size(H))

[('A', 'B', 'C'), ('A', 'D'), ('A', 'E', 'H')]
{'B', 'C', 'E', 'H'}
8


In [12]:
from hypergraphx.core.hypergraph import Hypergraph

def sub_of_size(hg: Hypergraph, size: int) -> Hypergraph:
    return hg.subhypergraph_by_orders(sizes=[size])

def is_uniform(hg: Hypergraph) -> bool:
    return hg.is_uniform()

In [38]:
H2 = sub_of_size(H, size=3)
print(H2)
print(is_uniform(H2))

Hypergraph with 8 nodes and 4 edges.
Distribution of hyperedge sizes: {3: 4}
True


In [39]:
from typing import Dict, Set
from hypergraphx.core.hypergraph import Hypergraph
from collections import defaultdict

def clique_projection_neighbors(hg: Hypergraph) -> Dict[str, Set[str]]:
    nbrs: Dict[str, Set[str]] = defaultdict(set)
    # Ensure all nodes appear as keys
    for u in hg.get_nodes():
        nbrs[str(u)]  # touch

    # For each node, every other member in an incident edge is a neighbor
    for u in hg.get_nodes():
        u = str(u)
        for e in hg.get_incident_edges(u):
            members = list(map(str, e))
            for v in members:
                if v != u:
                    nbrs[u].add(v)
                    nbrs[v].add(u)  # symmetry
    return {u: set(vs) for u, vs in nbrs.items()}

def clique_projection_degree(hg: Hypergraph) -> Dict[str, int]:
    nbrs = clique_projection_neighbors(hg)
    return {u: len(vs) for u, vs in nbrs.items()}

In [40]:
print(clique_projection_neighbors(H))
print(clique_projection_degree(H))

{'A': {'H', 'E', 'C', 'D', 'B'}, 'B': {'A', 'D', 'C'}, 'C': {'A', 'D', 'B'}, 'D': {'A', 'B', 'C'}, 'E': {'A', 'G', 'H', 'F'}, 'F': {'G', 'E'}, 'G': {'E', 'F'}, 'H': {'A', 'E'}}
{'A': 5, 'B': 3, 'C': 3, 'D': 3, 'E': 4, 'F': 2, 'G': 2, 'H': 2}


In [16]:
from typing import Dict, Set
from hypergraphx.core.hypergraph import Hypergraph

def edge_size_buckets(hg: Hypergraph) -> Dict[int, int]:
    return dict(sorted(hg.distribution_sizes().items()))

def nodes_covered_by_size(hg: Hypergraph) -> Dict[int, Set[str]]:
    out: Dict[int, Set[str]] = {}
    for e in hg.get_edges():
        s = len(e)
        out.setdefault(s, set()).update(map(str, e))
    return {k: set(sorted(v)) for k, v in sorted(out.items())}

def coverage_share_by_size(hg: Hypergraph) -> Dict[int, float]:
    total = float(hg.num_nodes()) if hg.num_nodes() else 1.0
    covered = nodes_covered_by_size(hg)
    return {s: len(v)/total for s, v in covered.items()}

def cumulative_coverage_share(hg: Hypergraph, ascending: bool = True) -> Dict[int, float]:
    sizes = sorted(edge_size_buckets(hg).keys(), reverse=not ascending)
    total = float(hg.num_nodes()) if hg.num_nodes() else 1.0
    seen: Set[str] = set()
    out: Dict[int, float] = {}
    by_size = nodes_covered_by_size(hg)
    for s in sizes:
        seen |= by_size.get(s, set())
        out[s] = len(seen)/total
    return out

In [17]:
print(edge_size_buckets(H))
print(nodes_covered_by_size(H))
print(coverage_share_by_size(H))
print(cumulative_coverage_share(H, ascending=True))

{2: 1, 3: 4}
{2: {'A', 'D'}, 3: {'E', 'F', 'A', 'G', 'B', 'H', 'C', 'D'}}
{2: 0.25, 3: 1.0}
{2: 0.25, 3: 1.0}


In [18]:
from typing import Set
from hypergraphx.core.hypergraph import Hypergraph

def neighbors_by_size(hg: Hypergraph, node: str, size: int) -> Set[str]:
    return set(map(str, hg.get_neighbors(node, size=size)))

In [19]:
print(neighbors_by_size(H, "A", size=3))

{'B', 'C', 'E', 'H'}


In [20]:
from typing import Dict, Set, Tuple
from hypergraphx.core.hypergraph import Hypergraph
from collections import defaultdict

EdgeKey = Tuple[str, ...]
Incidence = Dict[str, Set[EdgeKey]]

def _edge_key(e) -> EdgeKey:
    return tuple(sorted(map(str, e)))

def dual_nodes(hg: Hypergraph) -> Set[EdgeKey]:
    return { _edge_key(e) for e in hg.get_edges() }

def dual_incidence(hg: Hypergraph) -> Incidence:
    inc: Incidence = defaultdict(set)
    for e in hg.get_edges():
        ek = _edge_key(e)
        for u in map(str, e):
            inc[u].add(ek)
    return {u: set(sorted(v)) for u, v in sorted(inc.items())}

def dual_degrees(hg: Hypergraph) -> Dict[EdgeKey, int]:
    return { _edge_key(e): len(e) for e in hg.get_edges() }


In [21]:
print(dual_incidence(H))

{'A': {('A', 'B', 'C'), ('A', 'E', 'H'), ('A', 'D')}, 'B': {('A', 'B', 'C'), ('B', 'C', 'D')}, 'C': {('A', 'B', 'C'), ('B', 'C', 'D')}, 'D': {('B', 'C', 'D'), ('A', 'D')}, 'E': {('E', 'F', 'G'), ('A', 'E', 'H')}, 'F': {('E', 'F', 'G')}, 'G': {('E', 'F', 'G')}, 'H': {('A', 'E', 'H')}}


In [22]:
from typing import List, Tuple
from hypergraphx.core.hypergraph import Hypergraph
from itertools import combinations

EdgeKey = Tuple[str, ...]

def _ek(e) -> EdgeKey:
    return tuple(sorted(map(str, e)))

def _jaccard(a: set, b: set) -> float:
    u = len(a|b)
    return (len(a&b)/u) if u else 0.0

def topk_edge_jaccard(hg: Hypergraph, k: int) -> List[Tuple[EdgeKey, EdgeKey, float]]:
    edges = [set(map(str, e)) for e in hg.get_edges()]
    keys  = [_ek(e) for e in hg.get_edges()]
    triples: List[Tuple[EdgeKey, EdgeKey, float]] = []
    for (i, j) in combinations(range(len(edges)), 2):
        e1, e2 = keys[i], keys[j]
        if e2 < e1:  # enforce e1 < e2
            e1, e2 = e2, e1
            a, b = edges[j], edges[i]
        else:
            a, b = edges[i], edges[j]
        triples.append((e1, e2, _jaccard(a, b)))
    triples.sort(key=lambda t: (-t[2], t[0], t[1]))
    return triples[:k]

In [23]:
print(topk_edge_jaccard(H, 3))

[(('A', 'B', 'C'), ('B', 'C', 'D'), 0.5), (('A', 'B', 'C'), ('A', 'D'), 0.25), (('A', 'D'), ('A', 'E', 'H'), 0.25)]


In [24]:
from typing import Set, Tuple, List
from hypergraphx.core.hypergraph import Hypergraph

EdgeKey = Tuple[str, ...]

def _ek(e) -> EdgeKey:
    return tuple(sorted(map(str, e)))

def filter_edges_by_min_overlap(hg: Hypergraph, tau: int) -> Hypergraph:
    edges: List[Set[str]] = [set(map(str, e)) for e in hg.get_edges()]
    keep = [False]*len(edges)
    for i, Ei in enumerate(edges):
        best = 0
        for j, Ej in enumerate(edges):
            if i == j: continue
            best = max(best, len(Ei & Ej))
            if best >= tau:
                keep[i] = True
                break
    new_hg = Hypergraph()
    # Add only nodes that appear in kept edges
    kept_edges = [tuple(sorted(e)) for i, e in enumerate(edges) if keep[i]]
    if kept_edges:
        node_set = sorted(set().union(*kept_edges))
        new_hg.add_nodes(node_set)
        new_hg.add_edges(kept_edges)
    return new_hg

def remaining_edge_keys(hg: Hypergraph) -> Set[EdgeKey]:
    return { _ek(e) for e in hg.get_edges() }


In [25]:
print(filter_edges_by_min_overlap(H, 1))

Hypergraph with 8 nodes and 5 edges.
Distribution of hyperedge sizes: {3: 4, 2: 1}


In [26]:
from typing import List
from hypergraphx.core.hypergraph import Hypergraph

def components(hg: Hypergraph) -> List[List[str]]:
    """
    Return connected components as sorted lists of node labels (strings),
    using HGX's built-in connected components.
    """
    comps = hg.connected_components()  # HGX API
    return sorted([sorted(map(str, comp)) for comp in comps])

def largest_component_size(hg: Hypergraph) -> int:
    """
    Return the size (in nodes) of the largest connected component,
    using HGX's built-in method.
    """
    return int(hg.largest_component_size())  # HGX API

In [27]:
print(components(H))
print(largest_component_size(H))

[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']]
8


In [28]:
from typing import Dict
from math import log
from collections import defaultdict
from hypergraphx.core.hypergraph import Hypergraph

def participation_by_size(hg: Hypergraph) -> Dict[str, Dict[int, int]]:
    out: Dict[str, Dict[int, int]] = defaultdict(lambda: defaultdict(int))
    for e in hg.get_edges():
        s = len(e)
        for u in map(str, e):
            out[u][s] += 1
    # Normalize to plain dicts (sorted for determinism)
    return {u: {k: v for k, v in sorted(sz.items())} for u, sz in sorted(out.items())}

def participation_entropy(hg: Hypergraph) -> Dict[str, float]:
    part = participation_by_size(hg)
    ent: Dict[str, float] = {}
    for u, hist in part.items():
        total = sum(hist.values())
        if total == 0:
            ent[u] = 0.0
            continue
        H = 0.0
        for c in hist.values():
            p = c / total
            H -= p * log(p)
        ent[u] = H
    return ent


In [29]:
print(participation_by_size(H))

{'A': {2: 1, 3: 2}, 'B': {3: 2}, 'C': {3: 2}, 'D': {2: 1, 3: 1}, 'E': {3: 2}, 'F': {3: 1}, 'G': {3: 1}, 'H': {3: 1}}


In [30]:
from typing import Dict, Iterable, List
from hypergraphx.core.hypergraph import Hypergraph

def _degree_map(hg: Hypergraph) -> Dict[str, int]:
    return {str(u): hg.degree(u) for u in hg.get_nodes()}

def _rank_from_degrees(deg: Dict[str, int]) -> Dict[str, int]:
    order = sorted(deg.items(), key=lambda x: (-x[1], x[0]))  # degree desc, id asc
    return {u: i + 1 for i, (u, _) in enumerate(order)}

def degree_rank(hg: Hypergraph) -> Dict[str, int]:
    return _rank_from_degrees(_degree_map(hg))

def add_edge_and_rank_changes(hg: Hypergraph, new_edge: Iterable[str], top: int = 3) -> List[str]:
    # Degrees before
    deg0 = _degree_map(hg)

    # Degrees after: +1 for each unique node in the new edge (including new nodes)
    deg1 = deg0.copy()
    for u in set(map(str, new_edge)):    # dedupe just in case
        deg1[u] = deg1.get(u, 0) + 1

    # Ranks before/after
    r0 = _rank_from_degrees(deg0)
    r1 = _rank_from_degrees(deg1)

    # Compare on the union of nodes (so brand-new nodes are included)
    universe = sorted(set(deg1) | set(r0))
    diffs = [(u, abs(r0.get(u, len(universe)) - r1.get(u, len(universe)))) for u in universe]
    diffs.sort(key=lambda x: (-x[1], x[0]))  # biggest change first, tie-break by id

    return [u for u, _ in diffs[:top]]

In [31]:
print(degree_rank(H))

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8}


In [32]:
from typing import Tuple
from hypergraphx.readwrite import load_hypergraph

def load_counts(path: str) -> Tuple[int, int]:
    hg = load_hypergraph(path)
    return int(hg.num_nodes()), int(hg.num_edges())

In [33]:
dataset = 'datasets/coauth-cs-NeurIPS.json'
print(load_counts(dataset))

(12306, 7060)


In [34]:
from typing import List, Optional, Dict
from hypergraphx.core.hypergraph import Hypergraph
from hypergraphx.generation.random import random_hypergraph

SIZES = [2, 3, 4, 5, 6]

def _edges_by_size_for_majority(total_min: int, target_size: int) -> Dict[int, int]:
    """
    Build a histogram with a strict majority for `target_size` and totals >= total_min.
    We use a small baseline for non-target sizes and assign the rest to the target.
    """
    non_target = [s for s in SIZES if s != target_size]
    baseline = 1 * len(non_target)  # 1 edge for each non-target size
    # ensure target strictly more than any other; allocate a cushion above the minimum
    target_count = max(total_min - baseline, 1) + 2  # +2 cushion
    total = target_count + baseline
    hist = {s: (1 if s in non_target else target_count) for s in SIZES}
    # If we want to slightly exceed total_min to be robust to any generator de-duplication,
    # this histogram intentionally produces >= total_min edges.
    return hist

def generate_five_majority(n_min: int = 10, e_min: int = 10, seed: Optional[int] = None) -> List[Hypergraph]:
    hgs: List[Hypergraph] = []
    # Keep nodes exactly n_min (spec allows >=; tests only require >=)
    n = n_min
    for idx, target in enumerate(SIZES):
        edges_by_size = _edges_by_size_for_majority(e_min, target)
        sd = None if seed is None else seed + idx
        hg = random_hypergraph(num_nodes=n, num_edges_by_size=edges_by_size, seed=sd)
        hgs.append(hg)
    return hgs

In [35]:
HS = generate_five_majority(n_min=10, e_min=10, seed=42)
for hg in HS:
    print(hg)

Hypergraph with 10 nodes and 12 edges.
Distribution of hyperedge sizes: {2: 8, 3: 1, 4: 1, 5: 1, 6: 1}
Hypergraph with 10 nodes and 12 edges.
Distribution of hyperedge sizes: {2: 1, 3: 8, 4: 1, 5: 1, 6: 1}
Hypergraph with 10 nodes and 12 edges.
Distribution of hyperedge sizes: {2: 1, 3: 1, 4: 8, 5: 1, 6: 1}
Hypergraph with 10 nodes and 12 edges.
Distribution of hyperedge sizes: {2: 1, 3: 1, 4: 1, 5: 8, 6: 1}
Hypergraph with 10 nodes and 12 edges.
Distribution of hyperedge sizes: {2: 1, 3: 1, 4: 1, 5: 1, 6: 8}


In [36]:
from typing import Optional, Tuple, Set
from hypergraphx.core.hypergraph import Hypergraph
import random

def _is_connected(hg: Hypergraph) -> bool:
    try:
        return int(hg.largest_component_size()) == hg.num_nodes()
    except AttributeError:
        comps = hg.connected_components()
        return max((len(c) for c in comps), default=0) == hg.num_nodes()

def grow_until_connected(n: int = 20, seed: Optional[int] = None) -> Tuple[Hypergraph, int]:
    rng = random.Random(seed)

    hg = Hypergraph()
    nodes = [str(i) for i in range(n)]
    hg.add_nodes(nodes)

    seen: Set[frozenset] = set()  # frozensets of node labels

    while not _is_connected(hg):
        s = rng.randint(2, 4)          # edge size ∈ {2,3,4}
        members = tuple(sorted(rng.sample(nodes, s)))  # sample without replacement
        key = frozenset(members)
        if key in seen:
            continue
        seen.add(key)
        hg.add_edges([members])

        # (Optional) safety: avoid pathological loops
        if len(seen) > 50_000 and not _is_connected(hg):
            break

    return hg, hg.num_edges()

In [37]:
H_GROW, E_H_GROW = grow_until_connected(n=10, seed=42)
print(H_GROW)
print(E_H_GROW)

Hypergraph with 10 nodes and 12 edges.
Distribution of hyperedge sizes: {4: 4, 2: 6, 3: 2}
12
